In [1]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install dlib==19.18.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install face-recognition

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
#E.N.V.EDIRISINGHE

#Import libraries
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

#Import images and create a list of all the images
path = 'ImageAtendence'
images= []
className = []
myList =os.listdir(path)

#Import the images one by one
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    className.append(os.path.splitext(cl)[0])
print(className)

#Encoding Process
def findEncoding(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


#Mark Attendance
def markAttendence(name):
    with open('attend/Attendance.csv','r+') as f:
        #If somebody already arrived we don't want to repeat it
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            dtString = now.strftime("%m/%d/%Y, %H:%M:%S")
            f.writelines(f'\n{name},{dtString}')



encodeListKnow = findEncoding(images)

print('encoding complete')

#Initializing web camera
cap = cv2.VideoCapture(0)

#To get each frame one by one
while True:
    success, img = cap.read()
    imgc = cv2.resize(img,(0,0),None,0.25,0.25)
    imgc = cv2.cvtColor(imgc, cv2.COLOR_BGR2RGB)

    facesCurrentFrame = face_recognition.face_locations(imgc)
    encodeCurrentFrame = face_recognition.face_encodings(imgc,facesCurrentFrame)

    for encodeface,faceLoc in zip(encodeCurrentFrame,facesCurrentFrame):
        matches =face_recognition.compare_faces(encodeListKnow,encodeface)
        faceDic = face_recognition.face_distance(encodeListKnow,encodeface)
        #print(faceDic)
        matchindex = np.argmin(faceDic)


        if matches[matchindex]:


            name = className[matchindex].upper()
           # print(name)

            y1,x2,y2,x1 = faceLoc
            y1, x2, y2, x1 =y1*4,x2*4,y2*4,x1*4

            #Frame Formatting
            cv2.rectangle(img,(x1,y1),(x2,y2),(204,0,102),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(204,0,102),cv2.FILLED)
            #Font Formatting
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)

            markAttendence(name)

    cv2.imshow('webcam',img)
    cv2.waitKey(1)

['Ashan', 'Chris Evans', 'Chris Hemsworth', 'Kishore', 'Krishanth', 'Namal', 'Navoda', 'Rahul', 'Robert Downey', 'Shan', 'Thilakshika']
encoding complete


PermissionError: [Errno 13] Permission denied: 'attend/Attendance.csv'